### SCD2 using Apply Changes

In [0]:
project_name = 'scd2_dlt'

In [0]:
import dlt
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, lit, expr
from pyspark.sql import functions as f
from pyspark.sql.types import StructType, StructField, StringType, IntegerType,DateType

In [0]:
schema_csv = StructType([
    StructField('EmPId', IntegerType(),nullable=False),
    StructField('FirstName', StringType(),nullable=False),
    StructField('LastName', StringType(),nullable=False),
    StructField('CreatedOn', DateType(),nullable=False),
    StructField('ModifiedOn', DateType(),nullable=False)
])

cloud_file_options = {
    'cloudFiles.format': 'csv'
}

In [0]:
project_path = f'/FileStore/tables/{project_name}/data'
dbutils.fs.mkdirs(project_path)

True

In [0]:
@dlt.table(
    name='bronze_employee'
)
def bronze_load():
    df = spark.readStream.format('cloudFiles').options(**cloud_file_options).schema(schema_csv).load(project_path)
    df = df.withColumn('file_processed_date', F.date_format(F.current_timestamp(),'yyyy-MM--dd HH:mm:ss'))
    return df


In [0]:
dlt.create_streaming_table(
    name = 'silver_employee'
)

In [0]:
dlt.apply_changes(
    target='silver_employee',
    source='bronze_employee',
    keys=['EmPId'],
    stored_as_scd_type='1',
    sequence_by='file_processed_date'
)

In [0]:
?dlt.apply_changes

Signature: dlt.apply_changes(target, source, keys, sequence_by, where=None, ignore_null_updates=None, apply_as_deletes=None, apply_as_truncates=None, column_list=None, except_column_list=None, stored_as_scd_type='1', track_history_column_list=None, track_history_except_column_list=None, flow_name=None, once=False, ignore_null_updates_column_list=None, ignore_null_updates_except_column_list=None, columns_to_update=None)
Docstring:
Apply changes into the target table from the Change Data Capture (CDC) source. Target table must
have already been created using create_target_table function. Only one of column_list and
except_column_list can be specified. Also only one of track_history_column_list and
track_history_except_column_list can be specified.

Example:
apply_changes(
  target = "target",
  source = "source",
  keys = ["key"],
  sequence_by = "sequence_expr",
  ignore_null_updates = True,
  column_list = ["key", "value"],
  stored_as_scd_type = "1",
  track_history_column_list = ["va

### SCD1 Trigger

In [0]:
dlt.apply_changes(
    target='silver_employee',
    source='bronze_employee',
    keys=['EmPId'],
    stored_as_scd_type='1',
    sequence_by='file_processed_date',    
    flow_name=f'{project_name}_bronze2silver'
)

### SCD2 Trigger

In [0]:
dlt.apply_changes(
    target='silver_employee',
    source='bronze_employee',
    keys=['EmPId'],
    stored_as_scd_type='2',
    sequence_by='file_processed_date',
    track_history_column_list=['file_processed_date'],
    flow_name=f'{project_name}_bronze2silver'
)